In [ ]:
# Installing
%pip install psycopg2 numpy

# Connect to DB

In [3]:
import psycopg2
from psycopg2.extras import execute_values
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

user = os.getenv('USER')
host = os.getenv('HOST')
port = os.getenv('PORT')
password = os.getenv('PASSWORD')
dbname = os.getenv('DB1')

# Set up PostgreSQL connection
connection = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)
cursor = connection.cursor()


# Create table

In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS accounts (
    accNo SERIAL PRIMARY KEY,
    accName TEXT,
    nameEmbedding VECTOR(1536)  -- Adjust the dimension based on your embedding size
);
""")
connection.commit()


# Insert Embedding Data

In [ ]:
# Get embedding
from langchain.embeddings import OpenAIEmbeddings

openai_api = os.getenv('OPENAI_API_KEY')

embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api)

texts = ['Chứng khoán Chính phủ, chứng khoán chính quyền địa phương', 
         'Chiết khấu công cụ chuyển nhượng và giấy tờ có giá đối với các tổ chức kinh tế, cá nhân trong nước',
         'Cho vay vốn bằng đồng Việt Nam nhận trực tiếp của các Tổ chức Quốc tế',
         'Chứng từ có giá trị bằng ngoại tệ do nước ngoài gửi đến đợi thanh toán',
         'Giá trị tài sản cho thuê tài chính theo thời gian sử dụng']

# Embed batch
embeddings = embedder.embed_documents(texts = texts)

In [7]:
# Take 1 embedding to commit
embedding = embeddings[0]

cursor.execute("""
INSERT INTO accounts (accName, nameEmbedding)
VALUES (%s, %s);
""", ("Chứng khoán Chính phủ, chứng khoán chính quyền địa phương", embedding))

connection.commit()


# Query vector DB

- <-> - L2 distance
- <#> - (negative) inner product
- <=> - cosine distance
- <+> - L1 distance (added in 0.7.0)
- <~> - Hamming distance (binary vectors, added in 0.7.0)
- <%> - Jaccard distance (binary vectors, added in 0.7.0)


In [12]:
query_embedding = embedder.embed_documents(texts = ['Chứng khoán chính quyền địa phương'])
embedding_str = f"ARRAY{query_embedding[0]}::vector"

connection.rollback()

cursor.execute(f"""
    SELECT accNo, accName, nameEmbedding <-> {embedding_str} AS distance
    FROM accounts
    ORDER BY distance
    LIMIT 5;
""")

results = cursor.fetchall()

for row in results:
    print(row)


(1, 'Chứng khoán Chính phủ, chứng khoán chính quyền địa phương', 0.4800928087215958)


Output: (id, accName, L2 distance) <-> (1, 'Chứng khoán Chính phủ, chứng khoán chính quyền địa phương', 0.4800928087215958)